# Kerasによるモデル構築
- 年代(2010,2011,,,)を連続値として読み込んだもの
- 年代をダミー変数で読み込んだもの
- 年代を特徴量として読み込まなかったもの

## 必要なモジュールのimport

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization

Using TensorFlow backend.


## 必要なデータセットの準備

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")
train_df_y = train_df[["y"]]
train_df = train_df.drop(["y"],axis=1)
train_df_y.reset_index()
df = pd.concat([train_df,test_df])
df['datetime'] = pd.to_datetime(df.datetime)
#何月か
df['Month'] = df['datetime'].dt.month
#何日か
df['DayofMonth'] = df['datetime'].dt.day
#何曜日か
df['dayofweek'] = df['datetime'].dt.dayofweek
#何年か
df['year'] = df['datetime'].dt.year
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df["year"])

df["year_num"] = le.transform(df["year"])
#ダミー変数化
#何月か
month_df = pd.get_dummies(df['Month'], prefix='M', prefix_sep='_')
#何日か
dayofmonth_df = pd.get_dummies(df["DayofMonth"], prefix='DM', prefix_sep='_')
#何曜日か
dayofweek_df = pd.get_dummies(df['dayofweek'], prefix='DW', prefix_sep='_')
#何年か
year_df= pd.get_dummies(df['year'], prefix='y', prefix_sep='_')
#priceはどうか 
price_am_df = pd.get_dummies(df['price_am'], prefix='PA', prefix_sep='_')
price_pm_df = pd.get_dummies(df['price_pm'], prefix='PP', prefix_sep='_')
#datetimeを保存
df_id = df[['datetime']]
#year_df削除
df = pd.concat([df,
                month_df,
                dayofmonth_df,
                dayofweek_df,
               price_am_df,
               price_pm_df,
               year_df], axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]
test_df_datetime = test_df["datetime"]

In [7]:
df.columns

Index(['datetime', 'client', 'close', 'price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek', 'year', 'year_num', 'M_1', 'M_2', 'M_3',
       'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12',
       'DM_1', 'DM_2', 'DM_3', 'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9',
       'DM_10', 'DM_11', 'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17',
       'DM_18', 'DM_19', 'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25',
       'DM_26', 'DM_27', 'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1',
       'DW_2', 'DW_3', 'DW_4', 'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2',
       'PA_3', 'PA_4', 'PA_5', 'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4',
       'PP_5', 'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015',
       'y_2016', 'y_2017'],
      dtype='object')

# 年代(2010,2011,,,)を連続値として読み込んだモデル

In [96]:
df = df.drop(['datetime','price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek',"year_num"],axis=1)

In [97]:
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]
df.columns

Index(['client', 'close', 'year', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6',
       'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3',
       'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11',
       'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19',
       'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27',
       'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3',
       'DW_4', 'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4',
       'PA_5', 'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5'],
      dtype='object')

In [85]:
test_df.columns

Index(['client', 'close', 'year', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6',
       'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3',
       'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11',
       'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19',
       'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27',
       'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3',
       'DW_4', 'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4',
       'PA_5', 'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5'],
      dtype='object')

In [29]:
train_df

,year,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,M_9,...,PA_4,PA_5,PP_-1,PP_0,PP_1,PP_2,PP_3,PP_4,PP_5,y
0,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,17
1,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,18
2,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,20
3,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,20
4,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,14
5,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,14
6,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,4
7,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,10
8,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,12
9,2010,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,21


In [98]:
s = len(train_df.columns)
x = (train_df.iloc[:,:-1]).values
y = (train_df.iloc[:,-1:]).values
print("x",x)
print("y",y)
s

x [[   0    0 2010 ...,    0    0    0]
 [   0    0 2010 ...,    0    0    0]
 [   0    0 2010 ...,    0    0    0]
 ..., 
 [   1    0 2016 ...,    0    1    0]
 [   1    0 2016 ...,    0    1    0]
 [   1    0 2016 ...,    0    1    0]]
y [[ 17]
 [ 18]
 [ 20]
 ..., 
 [ 98]
 [ 99]
 [105]]


68

In [ ]:
model = Sequential()
model.add(Dense(140,input_dim=73,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(70,input_dim=73,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.05,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model.fit(x.values,y.values,nb_epoch=1000)

In [87]:
model = Sequential()
model.add(Dense(130,input_shape=(s-1,),activation='relu'))
model.add(Dense(65,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.05,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model.fit(x,y,nb_epoch=1000)
#15.1921528755

Epoch 1/1000
2101/2101 [==============================] - 0s - loss: 55.5956 - mean_absolute_error: 55.5956     
Epoch 2/1000
2101/2101 [==============================] - 0s - loss: 13.9624 - mean_absolute_error: 13.9624     
Epoch 3/1000
2101/2101 [==============================] - 0s - loss: 13.5678 - mean_absolute_error: 13.5678     
Epoch 4/1000
2101/2101 [==============================] - 0s - loss: 13.3212 - mean_absolute_error: 13.3212     
Epoch 5/1000
2101/2101 [==============================] - 0s - loss: 12.2701 - mean_absolute_error: 12.2701     
Epoch 6/1000
2101/2101 [==============================] - 0s - loss: 11.4691 - mean_absolute_error: 11.4691     
Epoch 7/1000
2101/2101 [==============================] - 0s - loss: 11.0049 - mean_absolute_error: 11.0049     
Epoch 8/1000
2101/2101 [==============================] - 0s - loss: 10.5440 - mean_absolute_error: 10.5440     
Epoch 9/1000
2101/2101 [==============================] - 0s - loss: 10.8626 - mean_absolute_err

2101/2101 [==============================] - 0s - loss: 9.9768 - mean_absolute_error: 9.9768       
Epoch 146/1000
2101/2101 [==============================] - 0s - loss: 9.3509 - mean_absolute_error: 9.3509     
Epoch 147/1000
2101/2101 [==============================] - 0s - loss: 9.5078 - mean_absolute_error: 9.5078     
Epoch 148/1000
2101/2101 [==============================] - 0s - loss: 9.1320 - mean_absolute_error: 9.1320     
Epoch 149/1000
2101/2101 [==============================] - 0s - loss: 9.4452 - mean_absolute_error: 9.4452     
Epoch 150/1000
2101/2101 [==============================] - 0s - loss: 9.5843 - mean_absolute_error: 9.5843     
Epoch 151/1000
2101/2101 [==============================] - 0s - loss: 9.1560 - mean_absolute_error: 9.1560     
Epoch 152/1000
2101/2101 [==============================] - 0s - loss: 9.2165 - mean_absolute_error: 9.2165     
Epoch 153/1000
2101/2101 [==============================] - 0s - loss: 9.0815 - mean_absolute_error: 9.0815  

2101/2101 [==============================] - 0s - loss: 9.3113 - mean_absolute_error: 9.3113     
Epoch 290/1000
2101/2101 [==============================] - 0s - loss: 9.1610 - mean_absolute_error: 9.1610     
Epoch 291/1000
2101/2101 [==============================] - 0s - loss: 9.0780 - mean_absolute_error: 9.0780     
Epoch 292/1000
2101/2101 [==============================] - 0s - loss: 9.2406 - mean_absolute_error: 9.2406     
Epoch 293/1000
2101/2101 [==============================] - 0s - loss: 9.1024 - mean_absolute_error: 9.1024     
Epoch 294/1000
2101/2101 [==============================] - 0s - loss: 9.3377 - mean_absolute_error: 9.3377     
Epoch 295/1000
2101/2101 [==============================] - 0s - loss: 9.2077 - mean_absolute_error: 9.2077     
Epoch 296/1000
2101/2101 [==============================] - 0s - loss: 9.0951 - mean_absolute_error: 9.0951     
Epoch 297/1000
2101/2101 [==============================] - 0s - loss: 9.0765 - mean_absolute_error: 9.0765    

2101/2101 [==============================] - 0s - loss: 9.0345 - mean_absolute_error: 9.0345     
Epoch 434/1000
2101/2101 [==============================] - 0s - loss: 9.1982 - mean_absolute_error: 9.1982     
Epoch 435/1000
2101/2101 [==============================] - 0s - loss: 9.1880 - mean_absolute_error: 9.1880     
Epoch 436/1000
2101/2101 [==============================] - 0s - loss: 9.0918 - mean_absolute_error: 9.0918     
Epoch 437/1000
2101/2101 [==============================] - 0s - loss: 9.4699 - mean_absolute_error: 9.4699     
Epoch 438/1000
2101/2101 [==============================] - 0s - loss: 9.0370 - mean_absolute_error: 9.0370     
Epoch 439/1000
2101/2101 [==============================] - 0s - loss: 9.0411 - mean_absolute_error: 9.0411     
Epoch 440/1000
2101/2101 [==============================] - 0s - loss: 9.0238 - mean_absolute_error: 9.0238     
Epoch 441/1000
2101/2101 [==============================] - 0s - loss: 9.0120 - mean_absolute_error: 9.0120    

2101/2101 [==============================] - 0s - loss: 9.0542 - mean_absolute_error: 9.0542     
Epoch 578/1000
2101/2101 [==============================] - 0s - loss: 9.0096 - mean_absolute_error: 9.0096     
Epoch 579/1000
2101/2101 [==============================] - 0s - loss: 9.0077 - mean_absolute_error: 9.0077     
Epoch 580/1000
2101/2101 [==============================] - 0s - loss: 9.0403 - mean_absolute_error: 9.0403     
Epoch 581/1000
2101/2101 [==============================] - 0s - loss: 9.1639 - mean_absolute_error: 9.1639     
Epoch 582/1000
2101/2101 [==============================] - 0s - loss: 9.3725 - mean_absolute_error: 9.3725     
Epoch 583/1000
2101/2101 [==============================] - 0s - loss: 9.0519 - mean_absolute_error: 9.0519     
Epoch 584/1000
2101/2101 [==============================] - 0s - loss: 9.0215 - mean_absolute_error: 9.0215     
Epoch 585/1000
2101/2101 [==============================] - 0s - loss: 9.0600 - mean_absolute_error: 9.0600    

2101/2101 [==============================] - 0s - loss: 9.1927 - mean_absolute_error: 9.1927     
Epoch 722/1000
2101/2101 [==============================] - 0s - loss: 9.0702 - mean_absolute_error: 9.0702     
Epoch 723/1000
2101/2101 [==============================] - 0s - loss: 8.9942 - mean_absolute_error: 8.9942     
Epoch 724/1000
2101/2101 [==============================] - 0s - loss: 9.0137 - mean_absolute_error: 9.0137     
Epoch 725/1000
2101/2101 [==============================] - 0s - loss: 9.1161 - mean_absolute_error: 9.1161     
Epoch 726/1000
2101/2101 [==============================] - 0s - loss: 9.1302 - mean_absolute_error: 9.1302     
Epoch 727/1000
2101/2101 [==============================] - 0s - loss: 9.0033 - mean_absolute_error: 9.0033     
Epoch 728/1000
2101/2101 [==============================] - 0s - loss: 9.0099 - mean_absolute_error: 9.0099     
Epoch 729/1000
2101/2101 [==============================] - 0s - loss: 9.1498 - mean_absolute_error: 9.1498    

2101/2101 [==============================] - 0s - loss: 9.0757 - mean_absolute_error: 9.0757     
Epoch 865/1000
2101/2101 [==============================] - 0s - loss: 9.0821 - mean_absolute_error: 9.0821     
Epoch 866/1000
2101/2101 [==============================] - 0s - loss: 8.9756 - mean_absolute_error: 8.9756     
Epoch 867/1000
2101/2101 [==============================] - 0s - loss: 9.1786 - mean_absolute_error: 9.1786     
Epoch 868/1000
2101/2101 [==============================] - 0s - loss: 8.9906 - mean_absolute_error: 8.9906     
Epoch 869/1000
2101/2101 [==============================] - 0s - loss: 8.9868 - mean_absolute_error: 8.9868     
Epoch 870/1000
2101/2101 [==============================] - 0s - loss: 9.0427 - mean_absolute_error: 9.0427     
Epoch 871/1000
2101/2101 [==============================] - 0s - loss: 8.9808 - mean_absolute_error: 8.9808     
Epoch 872/1000
2101/2101 [==============================] - 0s - loss: 9.1908 - mean_absolute_error: 9.1908    

In [89]:
test_df["y"] = model.predict(test_df.values)
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df_datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  se

In [80]:
test_df["y"] = model.predict(test_df.values)
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


KeyError: 'close'

In [ ]:
def reg_model():
    model = Sequential()
    model.add(Dense(10, input_dim=55, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    # compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# 年代をダミー変数で読み込んだモデル

In [3]:
df = pd.concat([df,
                month_df,
                dayofmonth_df,
                dayofweek_df,
               price_am_df,
               price_pm_df,
               year_df], axis=1)
df = df.drop(['datetime','price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek',"year_num","year"],axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [3]:
df = df.drop(['datetime','price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek',"year_num","year"],axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [3]:
#y_2017を除く
test_df_y_2017= test_df["y_2017"]
df = df.drop(['price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek',"year_num","year",'y_2017'],axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [55]:
df.columns

Index(['client', 'close', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7',
       'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3', 'DM_4',
       'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11', 'DM_12',
       'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19', 'DM_20',
       'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27', 'DM_28',
       'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3', 'DW_4',
       'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4', 'PA_5',
       'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5', 'y_2010',
       'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016'],
      dtype='object')

# 組み直すぜ

In [4]:
#2015-03-25から2016-03-31
train_df = train_df[train_df["close"]==0]
df_h = train_df[train_df["datetime"]>="2015-03-30"]
df_h = df_h.drop(['y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014'],axis=1)

In [5]:
df_h

,datetime,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,...,PP_-1,PP_0,PP_1,PP_2,PP_3,PP_4,PP_5,y_2015,y_2016,y
1733,2015-03-30,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,87
1734,2015-03-31,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,75
1735,2015-04-01,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,61
1736,2015-04-02,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,60
1737,2015-04-03,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,75
1738,2015-04-04,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,86
1739,2015-04-05,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,69
1740,2015-04-06,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,61
1741,2015-04-07,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,54
1742,2015-04-08,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,55


In [6]:
x = df_h.drop(["y","datetime"],axis=1)
y = df_h["y"]

In [1]:
model = Sequential()
model.add(Dense(140,input_dim=68,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.05,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model.fit(x.values,y.values,nb_epoch=1000)

NameError: name 'Sequential' is not defined

In [8]:
test_df.columns

Index(['datetime', 'client', 'close', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6',
       'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3',
       'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11',
       'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19',
       'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27',
       'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3',
       'DW_4', 'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4',
       'PA_5', 'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5',
       'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016'],
      dtype='object')

In [9]:
df_h_t = test_df.drop(['datetime','y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014'],axis=1)

In [10]:
df_h_t["y_2016"] = int(1)

In [11]:
df_h_t

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PA_5,PP_-1,PP_0,PP_1,PP_2,PP_3,PP_4,PP_5,y_2015,y_2016
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
5,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
6,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
8,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
9,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [13]:
model.predict(df_h_t.values)

array([[  67.04380798],
       [  54.11724472],
       [  83.08117676],
       [  65.57807922],
       [  52.92107773],
       [  46.69384766],
       [  53.17100143],
       [  56.38679504],
       [  74.6042099 ],
       [  79.58763885],
       [  48.61555481],
       [  44.34412384],
       [  49.90920258],
       [  45.03925323],
       [  52.20812225],
       [  62.26229858],
       [  64.10107422],
       [  38.4256897 ],
       [  32.22497559],
       [  46.92057037],
       [  43.89868927],
       [  68.79335785],
       [  68.25038147],
       [  53.13130188],
       [  60.65604019],
       [  50.55623627],
       [  49.85339737],
       [  61.72583771],
       [  94.15044403],
       [  68.17003632],
       [  45.22001648],
       [  40.77082062],
       [  53.51880264],
       [  43.92892456],
       [  30.81806564],
       [  28.47699547],
       [  42.15968323],
       [  24.03152847],
       [  29.20555115],
       [  29.47467422],
       [  31.15501213],
       [  33.275

In [14]:
#y_2017の処理をしていない
test_df["y"] = model.predict(df_h_t.values)
test_df["y"] = (1 - test_df['close']) * test_df.y
submission = pd.concat([test_df_datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
submission
#10.786779079

,datetime,y
0,2016-04-01,67.043808
1,2016-04-02,54.117245
2,2016-04-03,83.081177
3,2016-04-04,65.578079
4,2016-04-05,52.921078
5,2016-04-06,46.693848
6,2016-04-07,53.171001
7,2016-04-08,56.386795
8,2016-04-09,74.604210
9,2016-04-10,79.587639


In [16]:
#y_2017の処理をする
test_df = pd.concat([test_df,test_df_y_2017],axis=1)
test_df["y"] = model.predict(df_h_t.values)
test_df["y"] = (1 - test_df['close']) * test_df.y + (1 - test_df['close'])*(test_df["y_2017"])*6 
submission = pd.concat([test_df_datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
#9.90655278145

In [17]:
submission

,datetime,y
0,2016-04-01,67.043808
1,2016-04-02,54.117245
2,2016-04-03,83.081177
3,2016-04-04,65.578079
4,2016-04-05,52.921078
5,2016-04-06,46.693848
6,2016-04-07,53.171001
7,2016-04-08,56.386795
8,2016-04-09,74.604210
9,2016-04-10,79.587639


In [11]:

train_df = train_df[train_df["close"]==0]
x = (train_df.iloc[:,:-1]).values
y = (train_df.iloc[:,-1:]).values
s

75

In [10]:
train_df = train_df[train_df["close"]==0]
x = train_df.drop(["y"],axis=1)
y = train_df["y"]

In [9]:
x

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PP_4,PP_5,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017
0,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [11]:
model = Sequential()
model.add(Dense(40,input_dim=74,activation='relu'))
model.add(Dense(25,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.05,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model.fit(x.values,y.values,nb_epoch=1000)

Epoch 1/1000
2074/2074 [==============================] - 0s - loss: 10.1855 - mean_absolute_error: 10.1855     
Epoch 2/1000
2074/2074 [==============================] - 0s - loss: 6.6658 - mean_absolute_error: 6.6658     
Epoch 3/1000
2074/2074 [==============================] - 0s - loss: 6.3061 - mean_absolute_error: 6.3061     
Epoch 4/1000
2074/2074 [==============================] - 0s - loss: 6.2930 - mean_absolute_error: 6.2930     
Epoch 5/1000
2074/2074 [==============================] - 0s - loss: 5.8504 - mean_absolute_error: 5.8504     
Epoch 6/1000
2074/2074 [==============================] - 0s - loss: 5.5457 - mean_absolute_error: 5.5457     - ETA: 0s - loss: 5.3514 - mean_absolute_error: 5.
Epoch 7/1000
2074/2074 [==============================] - 0s - loss: 5.5304 - mean_absolute_error: 5.5304     
Epoch 8/1000
2074/2074 [==============================] - 0s - loss: 5.1461 - mean_absolute_error: 5.1461     
Epoch 9/1000
2074/2074 [==============================] - 0s

2074/2074 [==============================] - 0s - loss: 1.4302 - mean_absolute_error: 1.4302     
Epoch 146/1000
2074/2074 [==============================] - 0s - loss: 1.4414 - mean_absolute_error: 1.4414     
Epoch 147/1000
2074/2074 [==============================] - 0s - loss: 1.3976 - mean_absolute_error: 1.3976     
Epoch 148/1000
2074/2074 [==============================] - 0s - loss: 1.3565 - mean_absolute_error: 1.3565     
Epoch 149/1000
2074/2074 [==============================] - 0s - loss: 1.3409 - mean_absolute_error: 1.3409     
Epoch 150/1000
2074/2074 [==============================] - 0s - loss: 1.3780 - mean_absolute_error: 1.3780     - ETA: 0s - loss: 1.3689 - mean_absolute_error: 1.36
Epoch 151/1000
2074/2074 [==============================] - 0s - loss: 1.3928 - mean_absolute_error: 1.3928     
Epoch 152/1000
2074/2074 [==============================] - 0s - loss: 1.4564 - mean_absolute_error: 1.4564     
Epoch 153/1000
2074/2074 [==============================] -

2074/2074 [==============================] - 0s - loss: 0.9897 - mean_absolute_error: 0.9897     
Epoch 290/1000
2074/2074 [==============================] - 0s - loss: 0.9519 - mean_absolute_error: 0.9519     
Epoch 291/1000
2074/2074 [==============================] - 0s - loss: 0.9519 - mean_absolute_error: 0.9519     
Epoch 292/1000
2074/2074 [==============================] - 0s - loss: 0.9457 - mean_absolute_error: 0.9457     
Epoch 293/1000
2074/2074 [==============================] - 0s - loss: 0.9587 - mean_absolute_error: 0.9587     
Epoch 294/1000
2074/2074 [==============================] - 0s - loss: 0.9361 - mean_absolute_error: 0.9361     
Epoch 295/1000
2074/2074 [==============================] - 0s - loss: 0.9496 - mean_absolute_error: 0.9496     
Epoch 296/1000
2074/2074 [==============================] - 0s - loss: 0.9798 - mean_absolute_error: 0.9798     
Epoch 297/1000
2074/2074 [==============================] - 0s - loss: 0.9279 - mean_absolute_error: 0.9279    

2074/2074 [==============================] - 0s - loss: 0.7807 - mean_absolute_error: 0.7807     
Epoch 434/1000
2074/2074 [==============================] - 0s - loss: 0.7724 - mean_absolute_error: 0.7724     
Epoch 435/1000
2074/2074 [==============================] - 0s - loss: 0.7932 - mean_absolute_error: 0.7932     
Epoch 436/1000
2074/2074 [==============================] - 0s - loss: 0.7674 - mean_absolute_error: 0.7674     
Epoch 437/1000
2074/2074 [==============================] - 0s - loss: 0.7596 - mean_absolute_error: 0.7596     
Epoch 438/1000
2074/2074 [==============================] - 0s - loss: 0.7985 - mean_absolute_error: 0.7985     
Epoch 439/1000
2074/2074 [==============================] - 0s - loss: 0.7710 - mean_absolute_error: 0.7710     
Epoch 440/1000
2074/2074 [==============================] - 0s - loss: 0.7751 - mean_absolute_error: 0.7751     
Epoch 441/1000
2074/2074 [==============================] - 0s - loss: 0.8023 - mean_absolute_error: 0.8023    

2074/2074 [==============================] - 0s - loss: 0.7113 - mean_absolute_error: 0.7113     
Epoch 578/1000
2074/2074 [==============================] - 0s - loss: 0.6955 - mean_absolute_error: 0.6955     
Epoch 579/1000
2074/2074 [==============================] - 0s - loss: 0.6820 - mean_absolute_error: 0.6820     
Epoch 580/1000
2074/2074 [==============================] - 0s - loss: 0.6945 - mean_absolute_error: 0.6945     
Epoch 581/1000
2074/2074 [==============================] - 0s - loss: 0.6783 - mean_absolute_error: 0.6783     
Epoch 582/1000
2074/2074 [==============================] - 0s - loss: 0.6737 - mean_absolute_error: 0.6737     
Epoch 583/1000
2074/2074 [==============================] - 0s - loss: 0.6893 - mean_absolute_error: 0.6893     
Epoch 584/1000
2074/2074 [==============================] - 0s - loss: 0.6921 - mean_absolute_error: 0.6921     
Epoch 585/1000
2074/2074 [==============================] - 0s - loss: 0.6780 - mean_absolute_error: 0.6780    

2074/2074 [==============================] - 0s - loss: 0.6111 - mean_absolute_error: 0.6111     
Epoch 722/1000
2074/2074 [==============================] - 0s - loss: 0.6285 - mean_absolute_error: 0.6285     
Epoch 723/1000
2074/2074 [==============================] - 0s - loss: 0.6179 - mean_absolute_error: 0.6179     
Epoch 724/1000
2074/2074 [==============================] - 0s - loss: 0.6162 - mean_absolute_error: 0.6162     
Epoch 725/1000
2074/2074 [==============================] - 0s - loss: 0.6114 - mean_absolute_error: 0.6114     
Epoch 726/1000
2074/2074 [==============================] - 0s - loss: 0.6143 - mean_absolute_error: 0.6143     
Epoch 727/1000
2074/2074 [==============================] - 0s - loss: 0.6219 - mean_absolute_error: 0.6219     
Epoch 728/1000
2074/2074 [==============================] - 0s - loss: 0.6243 - mean_absolute_error: 0.6243     
Epoch 729/1000
2074/2074 [==============================] - 0s - loss: 0.6258 - mean_absolute_error: 0.6258    

2074/2074 [==============================] - 0s - loss: 0.5829 - mean_absolute_error: 0.5829     
Epoch 866/1000
2074/2074 [==============================] - 0s - loss: 0.5804 - mean_absolute_error: 0.5804     
Epoch 867/1000
2074/2074 [==============================] - 0s - loss: 0.5822 - mean_absolute_error: 0.5822     
Epoch 868/1000
2074/2074 [==============================] - 0s - loss: 0.5833 - mean_absolute_error: 0.5833     
Epoch 869/1000
2074/2074 [==============================] - 0s - loss: 0.5804 - mean_absolute_error: 0.5804     
Epoch 870/1000
2074/2074 [==============================] - 0s - loss: 0.5730 - mean_absolute_error: 0.5730     
Epoch 871/1000
2074/2074 [==============================] - 0s - loss: 0.5685 - mean_absolute_error: 0.5685     
Epoch 872/1000
2074/2074 [==============================] - 0s - loss: 0.5844 - mean_absolute_error: 0.5844     
Epoch 873/1000
2074/2074 [==============================] - 0s - loss: 0.5830 - mean_absolute_error: 0.5830    

In [47]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(X.y.values,model_1.predict(X.iloc[:,:-1].values))


1.6065787835554644

In [12]:
model_1 = Sequential()
model_1.add(Dense(222,input_dim=73,activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))
model_1.add(Dense(111,activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))
model_1.add(Dense(74,activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))
model_1.add(Dense(37,activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))
model_1.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.06,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model_1.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model_1.fit(x.values,y.values,nb_epoch=600,batch_size=300)

Epoch 1/600
2074/2074 [==============================] - 2s - loss: 32.5527 - mean_absolute_error: 32.5527     
Epoch 2/600
2074/2074 [==============================] - 0s - loss: 21.2780 - mean_absolute_error: 21.2780     
Epoch 3/600
2074/2074 [==============================] - 0s - loss: 10.9609 - mean_absolute_error: 10.9609     
Epoch 4/600
2074/2074 [==============================] - 0s - loss: 9.7065 - mean_absolute_error: 9.7065       
Epoch 5/600
2074/2074 [==============================] - 0s - loss: 8.2000 - mean_absolute_error: 8.2000     
Epoch 6/600
2074/2074 [==============================] - 0s - loss: 7.1542 - mean_absolute_error: 7.1542     
Epoch 7/600
2074/2074 [==============================] - 0s - loss: 7.2085 - mean_absolute_error: 7.2085     
Epoch 8/600
2074/2074 [==============================] - 0s - loss: 6.9011 - mean_absolute_error: 6.9011     
Epoch 9/600
2074/2074 [==============================] - 0s - loss: 6.6034 - mean_absolute_error: 6.6034     
Ep

2074/2074 [==============================] - 0s - loss: 3.8479 - mean_absolute_error: 3.8479     
Epoch 147/600
2074/2074 [==============================] - 0s - loss: 4.0378 - mean_absolute_error: 4.0378     
Epoch 148/600
2074/2074 [==============================] - 0s - loss: 3.8982 - mean_absolute_error: 3.8982     
Epoch 149/600
2074/2074 [==============================] - 0s - loss: 3.8996 - mean_absolute_error: 3.8996     
Epoch 150/600
2074/2074 [==============================] - 0s - loss: 3.8544 - mean_absolute_error: 3.8544     
Epoch 151/600
2074/2074 [==============================] - 0s - loss: 3.9567 - mean_absolute_error: 3.9567     
Epoch 152/600
2074/2074 [==============================] - 0s - loss: 3.9391 - mean_absolute_error: 3.9391     
Epoch 153/600
2074/2074 [==============================] - 0s - loss: 3.8142 - mean_absolute_error: 3.8142     
Epoch 154/600
2074/2074 [==============================] - 0s - loss: 3.8817 - mean_absolute_error: 3.8817     
Epoch 

2074/2074 [==============================] - 0s - loss: 3.1768 - mean_absolute_error: 3.1768     
Epoch 291/600
2074/2074 [==============================] - 0s - loss: 3.3066 - mean_absolute_error: 3.3066     
Epoch 292/600
2074/2074 [==============================] - 0s - loss: 3.3731 - mean_absolute_error: 3.3731     
Epoch 293/600
2074/2074 [==============================] - 0s - loss: 3.3236 - mean_absolute_error: 3.3236     
Epoch 294/600
2074/2074 [==============================] - 0s - loss: 3.3323 - mean_absolute_error: 3.3323     
Epoch 295/600
2074/2074 [==============================] - 0s - loss: 3.5193 - mean_absolute_error: 3.5193     
Epoch 296/600
2074/2074 [==============================] - 0s - loss: 3.3991 - mean_absolute_error: 3.3991     
Epoch 297/600
2074/2074 [==============================] - 0s - loss: 3.3083 - mean_absolute_error: 3.3083     
Epoch 298/600
2074/2074 [==============================] - 0s - loss: 3.3753 - mean_absolute_error: 3.3753     
Epoch 

2074/2074 [==============================] - 0s - loss: 2.8243 - mean_absolute_error: 2.8243     
Epoch 435/600
2074/2074 [==============================] - 0s - loss: 2.8357 - mean_absolute_error: 2.8357     
Epoch 436/600
2074/2074 [==============================] - 0s - loss: 2.7850 - mean_absolute_error: 2.7850     
Epoch 437/600
2074/2074 [==============================] - 0s - loss: 2.8426 - mean_absolute_error: 2.8426     
Epoch 438/600
2074/2074 [==============================] - 0s - loss: 2.9995 - mean_absolute_error: 2.9995     
Epoch 439/600
2074/2074 [==============================] - 0s - loss: 2.8499 - mean_absolute_error: 2.8499     
Epoch 440/600
2074/2074 [==============================] - 0s - loss: 2.8358 - mean_absolute_error: 2.8358     
Epoch 441/600
2074/2074 [==============================] - 0s - loss: 2.8878 - mean_absolute_error: 2.8878     
Epoch 442/600
2074/2074 [==============================] - 0s - loss: 2.8623 - mean_absolute_error: 2.8623     
Epoch 

2074/2074 [==============================] - 0s - loss: 2.6274 - mean_absolute_error: 2.6274     
Epoch 581/600
2074/2074 [==============================] - 0s - loss: 2.6992 - mean_absolute_error: 2.6992     
Epoch 582/600
2074/2074 [==============================] - 0s - loss: 2.6059 - mean_absolute_error: 2.6059     
Epoch 583/600
2074/2074 [==============================] - 0s - loss: 2.6663 - mean_absolute_error: 2.6663     
Epoch 584/600
2074/2074 [==============================] - 0s - loss: 2.5606 - mean_absolute_error: 2.5606     
Epoch 585/600
2074/2074 [==============================] - 0s - loss: 2.6744 - mean_absolute_error: 2.6744     
Epoch 586/600
2074/2074 [==============================] - 0s - loss: 2.8489 - mean_absolute_error: 2.8489     
Epoch 587/600
2074/2074 [==============================] - 0s - loss: 2.5770 - mean_absolute_error: 2.5770     
Epoch 588/600
2074/2074 [==============================] - 0s - loss: 2.6725 - mean_absolute_error: 2.6725     
Epoch 

In [160]:
train_df

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PP_5,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,17
1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,18
2,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,20
3,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,20
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,14
5,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,14
6,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,4
7,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,10
8,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,12
9,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,21


In [24]:
X = train_df[train_df["y_2016"]==1]


In [13]:
X


,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PP_5,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y
2013,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,24
2014,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,17
2015,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,15
2016,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,22
2017,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,34
2018,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,50
2019,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,52
2020,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,26
2021,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,32
2022,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,32


In [13]:
model_1.predict(test_df.values)

array([[ 70.58143616],
       [ 62.78202057],
       [ 72.16921997],
       [ 65.05545044],
       [ 57.12307739],
       [ 57.89093781],
       [ 57.75588989],
       [ 56.11882019],
       [ 65.45783997],
       [ 70.75114441],
       [ 47.99266815],
       [ 44.20335388],
       [ 44.44915771],
       [ 33.35359192],
       [ 48.29202271],
       [ 59.73421478],
       [ 60.83938599],
       [ 39.27988815],
       [ 30.85521698],
       [ 31.3277359 ],
       [ 30.91400719],
       [ 61.89709473],
       [ 53.6260376 ],
       [ 61.0883255 ],
       [ 62.34931946],
       [ 35.94006729],
       [ 57.67779541],
       [ 66.16220856],
       [ 80.64307404],
       [ 78.4835968 ],
       [ 56.92092133],
       [ 38.99659729],
       [ 38.06901169],
       [ 31.615242  ],
       [ 23.09232712],
       [ 41.0133667 ],
       [ 40.79503632],
       [ 28.28481293],
       [ 27.90859604],
       [ 29.17647934],
       [ 18.38367462],
       [ 29.41860962],
       [ 37.23519897],
       [ 46

In [48]:
test_df["y_y"] = model_1.predict(test_df.values)
test_df["y_y"] = (1 - test_df['close']) * test_df.y_y + (1 - test_df['close'])*(test_df["y_2017"])*14 
test_df["y_y"][test_df["y_2017"]==1].mean()

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


54.0669348610772

In [49]:
test_df["y_y"][test_df["y_2017"]==0].mean()

45.849050091830165

In [51]:
submission = pd.concat([test_df_datetime, test_df.y_y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)

In [50]:
test_df["y_y"]

0      75.997551
1      57.824463
2      69.194160
3      67.443649
4      58.772072
5      58.111866
6      57.761421
7      54.767944
8      61.827248
9      66.278603
10     44.640305
11     50.218834
12     51.777222
13     46.533524
14     52.843609
15     53.903687
16     57.956436
17     33.526234
18     30.655117
19     42.268307
20     34.257462
21     56.612572
22     55.664597
23     57.454750
24     57.079475
25     49.196503
26     59.382103
27     61.808151
28     94.604866
29     84.147453
         ...    
335    53.640953
336    67.321587
337    60.654175
338    62.197773
339    50.885014
340    49.071114
341    51.795555
342    50.140118
343    60.146244
344    69.487923
345    68.585785
346    66.641510
347    70.687737
348    69.941433
349    76.236580
350    79.736542
351    77.991432
352    81.605179
353    88.135208
354    77.219364
355    79.810875
356    84.812759
357    80.668739
358    82.245026
359    83.967972
360    85.883591
361    84.152458
362    98.4322

In [21]:
#y_2017の処理をする
test_df["y_y"] = model_1.predict(test_df.values)
test_df["y_y"] = (1 - test_df['close']) * test_df.y_y + (1 - test_df['close'])*(test_df["y_2017"])*12 
submission = pd.concat([test_df_datetime, test_df.y_y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
test_df["y_y"][test_df["y_2017"]==1].mean()
#11.056

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


52.79022740258111

In [126]:
test_df["y"] = model.predict(test_df.values)
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df_datetime, test_df.y], axis=1)
#submission.to_csv("submission.csv", index=False,header=False)

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [142]:

test_df["y"] = (1 - test_df['close']) * test_df.y

test_df["y"][test_df["y_2017"]==0].mean()
#14.8993999603

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


87.27978311018511

In [138]:
submission = pd.concat([test_df_datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)

In [129]:
test_df["y"][test_df["y_2017"]==1]

275    24.202549
276    12.604004
277    10.021465
278    20.418152
279     8.664471
280    16.171764
281    29.857273
282    33.412254
283    17.660400
284    14.255262
285    16.409868
286     7.601124
287    23.205761
288    30.400116
289    27.019142
290    26.666477
291    22.918066
292    22.578671
293    14.473106
294    30.961180
295    53.401230
296    32.762760
297    33.745316
298    31.542862
299    45.499264
300    32.934319
301    46.704418
302    45.678459
303    50.493015
304    36.218330
         ...    
335    29.467316
336    38.379173
337    35.990921
338    33.421047
339    35.013512
340    31.280735
341    35.344971
342    36.991482
343    31.873585
344    53.307121
345    49.913475
346    58.355846
347    58.878788
348    58.050022
349    65.459297
350    68.801117
351    62.079010
352    82.116386
353    77.079216
354    63.145432
355    63.022404
356    64.645210
357    64.888924
358    62.405178
359    72.241997
360    81.048500
361    71.952995
362    59.8852

In [144]:
test_df

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PP_5,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,134.742584
1,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,104.559143
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,126.269081
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,106.028885
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,92.755932
5,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,103.519890
6,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,108.668579
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,102.068810
8,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,139.872253
9,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,119.323013


In [59]:
#y_2017の処理をする
#test_df_y_2017= test_df["y_2017"]
test_df["y_y"] = model_2.predict(test_df.values)
test_df = pd.concat([test_df,test_df_y_2017],axis=1)
test_df["y_y"] = (1 - test_df['close']) * test_df.y_y + (1 - test_df['close'])*(test_df["y_2017"])*12 
submission = pd.concat([test_df_datetime, test_df.y_y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
test_df["y_y"][test_df["y_2017"]==1].mean()
#12.2336356183

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


50.42732275856866

In [155]:
test_df["y_y"][test_df["y_2017"]==0].mean()

51.541601292003286

In [151]:
test_df = test_df.drop(["y"],axis=1)
test_df

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PP_4,PP_5,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [56]:
from keras.callbacks import ModelCheckpoint
model_2 = Sequential()
model_2.add(Dense(296,input_dim=73,activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.2))
model_2.add(Dense(1))
opt = keras.optimizers.nadam()
model_2.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
#model_2.fit(x.values,y.values,nb_epoch=400,batch_size=500)
check = ModelCheckpoint("model.hdf5")

history = model_2.fit(x.values,y.values, nb_epoch=500,batch_size=600,callbacks=[check])
#2.2003

Epoch 1/500
2074/2074 [==============================] - 1s - loss: 34.4978 - mean_absolute_error: 34.4978     
Epoch 2/500
2074/2074 [==============================] - 0s - loss: 34.3332 - mean_absolute_error: 34.3332     
Epoch 3/500
2074/2074 [==============================] - 0s - loss: 34.1563 - mean_absolute_error: 34.1563     
Epoch 4/500
2074/2074 [==============================] - 0s - loss: 33.9770 - mean_absolute_error: 33.9770     
Epoch 5/500
2074/2074 [==============================] - 0s - loss: 33.7240 - mean_absolute_error: 33.7240     
Epoch 6/500
2074/2074 [==============================] - 0s - loss: 33.4306 - mean_absolute_error: 33.4306     
Epoch 7/500
2074/2074 [==============================] - 0s - loss: 33.0487 - mean_absolute_error: 33.0487     
Epoch 8/500
2074/2074 [==============================] - 0s - loss: 32.5723 - mean_absolute_error: 32.5723     
Epoch 9/500
2074/2074 [==============================] - 0s - loss: 31.9884 - mean_absolute_error: 31.98

2074/2074 [==============================] - 0s - loss: 2.5594 - mean_absolute_error: 2.5594     
Epoch 147/500
2074/2074 [==============================] - 0s - loss: 2.5335 - mean_absolute_error: 2.5335     
Epoch 148/500
2074/2074 [==============================] - 0s - loss: 2.5343 - mean_absolute_error: 2.5343     - ETA: 0s - loss: 2.5152 - mean_absolute_error: 2.515
Epoch 149/500
2074/2074 [==============================] - 0s - loss: 2.6127 - mean_absolute_error: 2.6127     
Epoch 150/500
2074/2074 [==============================] - 0s - loss: 2.5374 - mean_absolute_error: 2.5374     
Epoch 151/500
2074/2074 [==============================] - 0s - loss: 2.6122 - mean_absolute_error: 2.6122     
Epoch 152/500
2074/2074 [==============================] - 0s - loss: 2.5191 - mean_absolute_error: 2.5191     
Epoch 153/500
2074/2074 [==============================] - 0s - loss: 2.4675 - mean_absolute_error: 2.4675     
Epoch 154/500
2074/2074 [==============================] - 0s - l

2074/2074 [==============================] - 0s - loss: 2.2024 - mean_absolute_error: 2.2024     
Epoch 291/500
2074/2074 [==============================] - 0s - loss: 2.2412 - mean_absolute_error: 2.2412     
Epoch 292/500
2074/2074 [==============================] - 0s - loss: 2.1185 - mean_absolute_error: 2.1185     
Epoch 293/500
2074/2074 [==============================] - 0s - loss: 2.2897 - mean_absolute_error: 2.2897     
Epoch 294/500
2074/2074 [==============================] - 0s - loss: 2.2115 - mean_absolute_error: 2.2115     
Epoch 295/500
2074/2074 [==============================] - 0s - loss: 2.2336 - mean_absolute_error: 2.2336     
Epoch 296/500
2074/2074 [==============================] - 0s - loss: 2.1091 - mean_absolute_error: 2.1091     
Epoch 297/500
2074/2074 [==============================] - 0s - loss: 2.1773 - mean_absolute_error: 2.1773     
Epoch 298/500
2074/2074 [==============================] - 0s - loss: 2.1314 - mean_absolute_error: 2.1314     
Epoch 

2074/2074 [==============================] - 0s - loss: 1.9195 - mean_absolute_error: 1.9195     
Epoch 435/500
2074/2074 [==============================] - 0s - loss: 1.9218 - mean_absolute_error: 1.9218     
Epoch 436/500
2074/2074 [==============================] - 0s - loss: 1.9482 - mean_absolute_error: 1.9482     
Epoch 437/500
2074/2074 [==============================] - 0s - loss: 1.9750 - mean_absolute_error: 1.9750     
Epoch 438/500
2074/2074 [==============================] - 0s - loss: 1.9371 - mean_absolute_error: 1.9371     
Epoch 439/500
2074/2074 [==============================] - 0s - loss: 1.9817 - mean_absolute_error: 1.9817     
Epoch 440/500
2074/2074 [==============================] - 0s - loss: 1.9515 - mean_absolute_error: 1.9515     
Epoch 441/500
2074/2074 [==============================] - 0s - loss: 2.0505 - mean_absolute_error: 2.0505     
Epoch 442/500
2074/2074 [==============================] - 0s - loss: 1.9914 - mean_absolute_error: 1.9914     
Epoch 

In [58]:
model_2.predict(test_df.values)

array([[ 75.26985168],
       [ 67.85611725],
       [ 74.33126831],
       [ 60.9234581 ],
       [ 50.27220154],
       [ 57.2989006 ],
       [ 56.21510315],
       [ 44.82242966],
       [ 72.36981964],
       [ 69.42378235],
       [ 42.25111771],
       [ 48.40825653],
       [ 49.78928375],
       [ 41.07395172],
       [ 63.02453995],
       [ 59.4355545 ],
       [ 49.21126556],
       [ 38.61985779],
       [ 40.53994751],
       [ 42.38869095],
       [ 36.93172073],
       [ 55.77327347],
       [ 63.26162338],
       [ 59.37261963],
       [ 59.50296021],
       [ 51.80044556],
       [ 63.89757538],
       [ 64.30220795],
       [ 88.2397995 ],
       [ 77.90835571],
       [ 56.21286011],
       [ 38.44210815],
       [ 39.44086838],
       [ 38.11177826],
       [ 34.80296326],
       [ 36.72237015],
       [ 47.71240997],
       [ 21.76734161],
       [ 32.85745239],
       [ 26.22276688],
       [ 27.97712326],
       [ 35.24021149],
       [ 41.89043808],
       [ 53

In [57]:
X = train_df[train_df["y_2016"]==1]
from sklearn.metrics import mean_absolute_error
mean_absolute_error(X.y.values,model_2.predict(X.iloc[:,:-1].values))

2.38619570679717

In [39]:
X.y

2013     24
2014     17
2015     15
2016     22
2017     34
2018     50
2019     52
2020     26
2021     32
2022     32
2023     22
2024     34
2025     47
2026     51
2027     20
2028     24
2029     36
2030     31
2031     43
2032     58
2033     42
2034     46
2035     42
2036     41
2037     53
2038     68
2039     78
2040     79
2041     50
2042     31
       ... 
2071     59
2072     53
2073     62
2074     67
2075     59
2076     53
2077     46
2078     56
2079     60
2080     66
2081     84
2082     77
2083     69
2084     81
2085     78
2086     87
2087    102
2088     94
2089    101
2090    100
2091     96
2092     96
2093     94
2094     90
2095     93
2096     94
2097     86
2098     98
2099     99
2100    105
Name: y, Length: 88, dtype: int64

In [31]:
from keras.callbacks import ModelCheckpoint
check = ModelCheckpoint("model.hdf5")
history = model_2.fit(x.values,y.values, nb_epoch=500, 
                    validation_split=0.2, batch_size=32,
                    callbacks=[check])

Train on 1659 samples, validate on 415 samples
Epoch 1/500
1659/1659 [==============================] - 0s - loss: 3.7267 - mean_absolute_error: 3.7267 - val_loss: 4.5472 - val_mean_absolute_error: 4.5472
Epoch 2/500
1659/1659 [==============================] - 0s - loss: 3.2235 - mean_absolute_error: 3.2235 - val_loss: 2.9953 - val_mean_absolute_error: 2.9953
Epoch 3/500
1659/1659 [==============================] - 0s - loss: 3.1191 - mean_absolute_error: 3.1191 - val_loss: 3.0047 - val_mean_absolute_error: 3.0047
Epoch 4/500
1659/1659 [==============================] - 0s - loss: 3.3462 - mean_absolute_error: 3.3462 - val_loss: 2.8471 - val_mean_absolute_error: 2.8471
Epoch 5/500
1659/1659 [==============================] - 0s - loss: 3.2495 - mean_absolute_error: 3.2495 - val_loss: 2.5730 - val_mean_absolute_error: 2.5730
Epoch 6/500
1659/1659 [==============================] - 0s - loss: 3.5874 - mean_absolute_error: 3.5874 - val_loss: 2.6019 - val_mean_absolute_error: 2.6019
Epoch

1659/1659 [==============================] - 0s - loss: 2.9584 - mean_absolute_error: 2.9584 - val_loss: 5.5482 - val_mean_absolute_error: 5.5482
Epoch 104/500
1659/1659 [==============================] - 0s - loss: 2.7032 - mean_absolute_error: 2.7032 - val_loss: 5.4302 - val_mean_absolute_error: 5.4302
Epoch 105/500
1659/1659 [==============================] - 0s - loss: 2.9973 - mean_absolute_error: 2.9973 - val_loss: 5.4892 - val_mean_absolute_error: 5.4892
Epoch 106/500
1659/1659 [==============================] - 0s - loss: 3.0528 - mean_absolute_error: 3.0528 - val_loss: 5.7647 - val_mean_absolute_error: 5.7647
Epoch 107/500
1659/1659 [==============================] - 0s - loss: 2.8870 - mean_absolute_error: 2.8870 - val_loss: 5.5477 - val_mean_absolute_error: 5.5477
Epoch 108/500
1659/1659 [==============================] - 0s - loss: 2.9220 - mean_absolute_error: 2.9220 - val_loss: 5.5830 - val_mean_absolute_error: 5.5830
Epoch 109/500
1659/1659 [=============================

1659/1659 [==============================] - 0s - loss: 2.7911 - mean_absolute_error: 2.7911 - val_loss: 6.4715 - val_mean_absolute_error: 6.4715
Epoch 206/500
1659/1659 [==============================] - 0s - loss: 2.5912 - mean_absolute_error: 2.5912 - val_loss: 6.4769 - val_mean_absolute_error: 6.4769
Epoch 207/500
1659/1659 [==============================] - 0s - loss: 3.1143 - mean_absolute_error: 3.1143 - val_loss: 6.3326 - val_mean_absolute_error: 6.3326
Epoch 208/500
1659/1659 [==============================] - 0s - loss: 2.5970 - mean_absolute_error: 2.5970 - val_loss: 6.0963 - val_mean_absolute_error: 6.0963
Epoch 209/500
1659/1659 [==============================] - 0s - loss: 2.8973 - mean_absolute_error: 2.8973 - val_loss: 6.4351 - val_mean_absolute_error: 6.4351
Epoch 210/500
1659/1659 [==============================] - 0s - loss: 2.8197 - mean_absolute_error: 2.8197 - val_loss: 6.8374 - val_mean_absolute_error: 6.8374
Epoch 211/500
1659/1659 [=============================

1659/1659 [==============================] - 0s - loss: 2.7067 - mean_absolute_error: 2.7067 - val_loss: 7.2100 - val_mean_absolute_error: 7.2100
Epoch 308/500
1659/1659 [==============================] - 0s - loss: 2.8576 - mean_absolute_error: 2.8576 - val_loss: 6.8373 - val_mean_absolute_error: 6.8373
Epoch 309/500
1659/1659 [==============================] - 0s - loss: 2.9721 - mean_absolute_error: 2.9721 - val_loss: 6.6025 - val_mean_absolute_error: 6.6025
Epoch 310/500
1659/1659 [==============================] - 0s - loss: 2.6042 - mean_absolute_error: 2.6042 - val_loss: 6.9807 - val_mean_absolute_error: 6.9807
Epoch 311/500
1659/1659 [==============================] - 0s - loss: 2.7039 - mean_absolute_error: 2.7039 - val_loss: 6.5485 - val_mean_absolute_error: 6.5485
Epoch 312/500
1659/1659 [==============================] - 0s - loss: 2.7925 - mean_absolute_error: 2.7925 - val_loss: 6.8586 - val_mean_absolute_error: 6.8586
Epoch 313/500
1659/1659 [=============================

1659/1659 [==============================] - 0s - loss: 2.9056 - mean_absolute_error: 2.9056 - val_loss: 7.3836 - val_mean_absolute_error: 7.3836
Epoch 410/500
1659/1659 [==============================] - 0s - loss: 2.8948 - mean_absolute_error: 2.8948 - val_loss: 7.1041 - val_mean_absolute_error: 7.1041
Epoch 411/500
1659/1659 [==============================] - 0s - loss: 2.7399 - mean_absolute_error: 2.7399 - val_loss: 7.0094 - val_mean_absolute_error: 7.0094
Epoch 412/500
1659/1659 [==============================] - 0s - loss: 2.5290 - mean_absolute_error: 2.5290 - val_loss: 7.3519 - val_mean_absolute_error: 7.3519
Epoch 413/500
1659/1659 [==============================] - 0s - loss: 2.9952 - mean_absolute_error: 2.9952 - val_loss: 7.5396 - val_mean_absolute_error: 7.5396
Epoch 414/500
1659/1659 [==============================] - 0s - loss: 2.6912 - mean_absolute_error: 2.6912 - val_loss: 7.1357 - val_mean_absolute_error: 7.1357
Epoch 415/500
1659/1659 [=============================

In [35]:
import matplotlib.pyplot as plt

def plot_history(history):
    # 精度の履歴をプロット
    plt.plot(history.history['acc'],"o-",label="accuracy")
    plt.plot(history.history['val_acc'],"o-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()

    # 損失の履歴をプロット
    plt.plot(history.history['loss'],"o-",label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='lower right')
    plt.show()
# modelに学習させた時の変化の様子をplot
plot_history(history)

KeyError: 'acc'

# 年代を特徴量として読み込まなかったモデル

In [23]:
df = pd.concat([df,
                month_df,
                dayofmonth_df,
                dayofweek_df,
               price_am_df,
               price_pm_df,], axis=1)
df = df.drop(['datetime', 'client', 'close', 'price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek',"year_num","year"],axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [24]:
df.columns

Index(['M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10',
       ...
       'PA_3', 'PA_4', 'PA_5', 'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4',
       'PP_5'],
      dtype='object', length=128)